### 回调处理器

#### 为什么要用回调处理器？
- 模块化和可重用性：例如，如果你有多个应用都需要流式输出到WebSocket，那么使用一个统一的`WebSocketStreamingHandler`可以避免可重复的代码。
- 灵活性：回调处理器提供了一种结构化的方式来响应各种时间，而不仅仅是流式输出。这意味着你可以为各种事件（如链开始、结束、错误发生等）定义特动的逻辑。
- 与LangChain框架中的其他组件紧密集成：回调处理器是为LangChain框架的特别设计，确保其内部机制的兼容性和高效性。
- 代码清晰且具有维护性：通过使用专门的回调处理器，你的代码结构会更清晰。当其他开发者查看或维护你的代码时，他们可以轻松地找到和理解回调逻辑。

### 回调处理器的工作流程
- `run_manager`用于管理和触发回调事件
- `callbacks=[]`表示的是为`run_manager`提供的回调处理器列表

#### 回调处理器的四个过程
- 开始执行链组件：此时，系统开始执行链组件，并为处理输入数据做好准备。然后检查`callbakcs=[]`参数，若传递了`callbacks=[]`，则系统进入下一步初始化`run_manager`。此时，系统使用传递的`callbacks=[]`列表来初始化`run_manager`，确保它包含了所有提供的回调参数器。
- 链组件执行中阶段：系统根据输入数据执行链组件。在各个关键节点，如数据处理、模型调用等，系统会检查`run_manager`是否需要触发任何回调。
- 触发回调截断：如果`run_manager`在某个执行阶段检测到需要触发的回调事件，它会按照`callbacks=[]`中定义的顺序触发回调处理器。
- 完成阶段：当所有任务都完成，并且所有必要的回调都被触发后，链组件的执行就此结束。

![回调工作流](./CallBackWorkFlow.png)

### LangChain中内置的回调处理器

In [1]:
# 导入必要的组件与函数
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [6]:
# 加载LLM，工具和智能体
llm = OpenAI()
tools = load_tools(["serpapi"],llm=llm)
agent = initialize_agent(
    llm = llm,
    tools = tools,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)



In [7]:
agent.run("在Quake中的快速算术平方根是如何实现的？")
# 由于此时智能体开启了verbose，因此会调用StdCallbackProcessor
# 并显示模型的推理过程



> Entering new AgentExecutor chain...
 研究Quake游戏的开发者和技术实现可能会提供有用的信息。
Action: Search
Action Input: "Quake game developer and technical implementation"
Observation: ['Quake is a first-person shooter game developed by id Software and published by GT Interactive. The first game in the Quake series, it was originally ...', 'This page documents information about a third-party game running on a non-Valve engine. This information is documented here because it has ...', 'The Quake engine is the game engine developed by id Software to power their 1996 video game Quake. It featured true 3D real-time rendering.', "If you want to be a game programmer, or for that matter any sort of programmer at all, here's the secret to success in just two words: Ship it.", "Planning and implementing Quake's pipeline was the biggest technical ... Amateur game developer ... technology, are we going to innovate in our game ...", "Let's play a game. Suppose that you are a software developer and have just been hired

'The Quake game engine was developed specifically for the game by id Software, with John Carmack being the primary programmer for the original version. It has since been modified and expanded upon by various developers over the years.'

In [8]:
# 现在构建一个带有verbose的智能体
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

# 创建LLM链
llm_chain = LLMChain(
    llm = OpenAI(),
    prompt = PromptTemplate.from_template("{input}")
)

llm_chain.run("沈阳的著名旅游景点有哪些？")

'\n\n1. 沈阳故宫：被誉为“北方的故宫”，是清代皇宫建筑的杰作，也是世界上保存最完整的皇家建筑群之一。\n\n2. 沈阳市中心街：又称“中街”，是沈阳市最繁华的商业街，拥有众多历史建筑和现代商业设施，是游客购物和品尝美食的热门地点。\n\n3. 朝阳湖公园：是沈阳市最大的城市公园，也是沈阳市民休闲娱乐的主要场所，湖光山色、花木扶疏，是赏花、散步、垂钓的好去处。\n\n4. 沈阳森林动物园：位于沈阳市北部，是中国最大的城市'

In [10]:
# 导入StdCallbackHandler以监听llm_chain的输出
from langchain.callbacks import StdOutCallbackHandler
handler = StdOutCallbackHandler()

llm_chain = LLMChain(
    llm = OpenAI(),
    prompt = PromptTemplate.from_template("{input}"),
    callbacks=[handler]
)

llm_chain.run("沈阳的著名旅游景点有哪些？")



> Entering new LLMChain chain...
Prompt after formatting:
沈阳的著名旅游景点有哪些？

> Finished chain.


'\n\n1. 沈阳故宫\n2. 北陵公园\n3. 东陵公园\n4. 凌河公园\n5. 沈阳植物园\n6. 沈阳森林动物园\n7. 沈阳世博园\n8. 现代汽车博物馆\n9. 沈阳市博物馆\n10. 沈阳草莓主题公园\n11. 长白山风景名胜区\n12. 沈阳八一罗马广场\n13. 沈阳市图书馆\n14. 沈阳博物院\n15. 沈阳大悦城\n16. 沈阳海洋极地世界\n17. 沈阳悦泉国际水城\n18. 沈阳皇家度假村\n19. 沈阳太阳岛风景'